# Exception Handling Example

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [1]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

Imports essential modules from CrewAI and initializes two tools, one for web scraping and one for search engine interaction.Also, sets up environment variables to securely store API keys required for accessing external services. 

In [6]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import os

# Create a search tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

import os
os.environ["OPENAI_API_KEY"] = 'aa'
os.environ["SERPER_API_KEY"] = 'aa'

Defines two agents with specific roles and goals, however both are forbidden to choose a topic on their own, and are required to ask each other until one of us provides the topic.

In [3]:
# Data Researcher Agent using Gemini and SerperSearch
article_researcher=Agent(
    role="Senior Researcher",
    goal='Unccover details regarding the {topic}. If no topic is given, then wait for it to be provided by the user or ask the other agent. Do not choose a topic on your own!',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of"
        "research, eager to explore and share knowledge about the topic. However you are unable to think a topic on your own and need specific topics before you can research"
    ),
    tools=[search_tool,]

)

# Article Writer Agent using GPT
article_writer = Agent(
  role='Senior Writer',
  goal='Narrate compelling tech stories about {topic}. If no topic is given, then wait for it to be provided by the user or ask the other agent. Do not choose a topic on your own!',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner. However you are unable to think a topic on your own and need specific topics before you can write."
  ),
  allow_delegation=True
)

Defines tasks for research and writing. 

In [4]:
# Research Task
research_task = Task(
    description=(
        "Conduct a thorough analysis on the given {topic}."
        "Utilize the search tool for any necessary online research."
        "Summarize key findings in a detailed report."
    ),
    expected_output='A detailed report on the data analysis with key insights.',
    tools=[search_tool, scrape_tool],
    agent=article_researcher,
)

# Writing Task
writing_task = Task(
    description=(
        "Write an insightful article based on the data analysis report. "
        "The article should be clear, engaging, and easy to understand."
    ),
    expected_output='A 6-paragraph article summarizing the data insights.',
    agent=article_writer,
)

Creates a crew and then kicks off the project. However, you can see it was stuck in a loop and we had to interupt the process manually.

In [5]:
from crewai import Crew, Process

# Form the crew and define the process
crew = Crew(
    agents=[article_researcher, article_writer],
    tasks=[research_task, writing_task],
    process=Process.sequential
)

research_inputs = {
    'topic': ''
}

# Kick off the crew
result = crew.kickoff(inputs=research_inputs)

# Agent: Senior Researcher
## Task: Conduct a thorough analysis on the given .Utilize the search tool for any necessary online research.Summarize key findings in a detailed report.


# Agent: Senior Researcher
## Final Answer: 
I cannot proceed without a specific topic to analyze. Please provide a topic, or I can check with the other agent for details.


# Agent: Senior Writer
## Task: Write an insightful article based on the data analysis report. The article should be clear, engaging, and easy to understand.
# Agent: Senior Researcher
## Task: What specific topic should I analyze to write an insightful article based on a data analysis report?


# Agent: Senior Researcher
## Thought: I need a specific topic related to data analysis to proceed with the research. I will ask for suggestions on compelling subjects that could be insightful for the article.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"current trends in data analysis topics for engaging